## Bias Mitigation using Fairlearn - Heart Failure Prediction Dataset (Source: https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction/data)

In [1]:
#load preprocessed data 
import pandas as pd
train_df = pd.read_csv("./data_subsets/train_50_50.csv")

X_test = pd.read_csv("./data_splits/X_test.csv")
y_test = pd.read_csv("./data_splits/y_test.csv")

#check out the data
train_df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,61,1,3,146.0,241.0,0,0,148.0,1,3.0,0,1
1,39,1,1,130.0,215.0,0,0,120.0,0,0.0,2,0
2,60,0,0,150.0,240.0,0,0,171.0,0,0.9,2,0
3,49,1,3,128.0,212.0,0,0,96.0,1,0.0,1,1
4,50,0,2,140.0,288.0,0,0,140.0,1,0.0,1,1


In [2]:
# Ensure y_test is a Series (not a DataFrame with 1 column)
y_test = y_test.squeeze("columns")

# Define target and sensitive column names
TARGET = "HeartDisease"
SENSITIVE = "Sex"

# Split train into X/y
X_train = train_df.drop(columns=[TARGET])
y_train = train_df[TARGET]

# Extract sensitive features separately
A_train = X_train[SENSITIVE].astype(int)
A_test  = X_test[SENSITIVE].astype(int)


In [3]:
TARGET = "HeartDisease"
SENSITIVE = "Sex"   # 1 = Male, 0 = Female

categorical_cols = ['Sex','ChestPainType','FastingBS','RestingECG','ExerciseAngina','ST_Slope']
continuous_cols  = ['Age','RestingBP','Cholesterol','MaxHR','Oldpeak']

In [4]:
# Split train into X / y and keep sensitive feature for fairness evaluation
X_train = train_df.drop(columns=[TARGET])
y_train = train_df[TARGET]

In [5]:
# scale numeric features only, fit on train, transform test
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_num_scaled = pd.DataFrame(
    scaler.fit_transform(X_train[continuous_cols]),
    columns=continuous_cols, index=X_train.index
)
X_test_num_scaled = pd.DataFrame(
    scaler.transform(X_test[continuous_cols]),
    columns=continuous_cols, index=X_test.index
)

In [6]:
#one-hot encode categoricals; numeric are kept as is 
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown="ignore", drop="if_binary", sparse_output=False)
ohe.fit(X_train[categorical_cols])

X_train_cat = pd.DataFrame(
    ohe.transform(X_train[categorical_cols]),
    columns=ohe.get_feature_names_out(categorical_cols),
    index=X_train.index
)
X_test_cat = pd.DataFrame(
    ohe.transform(X_test[categorical_cols]),
    columns=ohe.get_feature_names_out(categorical_cols),
    index=X_test.index
)

In [7]:
# Assemble final matrices
X_train_ready = pd.concat([X_train_cat, X_train_num_scaled], axis=1)
X_test_ready  = pd.concat([X_test_cat,  X_test_num_scaled],  axis=1)

print("Final feature shapes:", X_train_ready.shape, X_test_ready.shape)

Final feature shapes: (600, 18) (184, 18)


### Traditional ML Models - Baseline: K-Nearest Neighbors (KNN) & Decision Tree (DT)

In [8]:
#import required libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)

#define a function 
def evaluate_model(y_true, y_pred, model_name):
    print(f"=== {model_name} Evaluation ===")
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, average='binary'))
    print("Recall   :", recall_score(y_true, y_pred, average='binary'))
    print("F1 Score :", f1_score(y_true, y_pred, average='binary'))
    print("\nClassification Report:\n", classification_report(y_true, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
    print("\n" + "="*40 + "\n")

### PCA-KNN

In [9]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)
import numpy as np

#1) PCA + KNN pipeline (on one-hot encoded + scaled features)
pca_knn = Pipeline([
    ('pca', PCA(n_components=0.95, random_state=42)),  # keep 95% variance
    ('knn', KNeighborsClassifier(
        n_neighbors=15, metric='manhattan', weights='distance'
    ))
])

pca_knn.fit(X_train_ready, y_train)

# Inspect PCA details
n_comp = pca_knn.named_steps['pca'].n_components_
expl_var = pca_knn.named_steps['pca'].explained_variance_ratio_.sum()

print("=== Baseline (tuned PCA+KNN, no mitigation) ===")
# 2) Evaluate 
y_pred_pca_knn = pca_knn.predict(X_test_ready)
probs_pca_knn = pca_knn.predict_proba(X_test_ready)[:, 1]
  
evaluate_model(y_test, y_pred_pca_knn, "KNN (best params)")

=== Baseline (tuned PCA+KNN, no mitigation) ===
=== KNN (best params) Evaluation ===
Accuracy : 0.875
Precision: 0.9247311827956989
Recall   : 0.8431372549019608
F1 Score : 0.882051282051282

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.91      0.87        82
           1       0.92      0.84      0.88       102

    accuracy                           0.88       184
   macro avg       0.87      0.88      0.87       184
weighted avg       0.88      0.88      0.88       184

Confusion Matrix:
 [[75  7]
 [16 86]]




### Post-Processing -  KNN

In [10]:
# Demographic Parity post-processing for your tuned PCA+KNN

from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.metrics import (
    MetricFrame, true_positive_rate, false_positive_rate, selection_rate,
    demographic_parity_difference, equalized_odds_difference
)
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

# Helper function
def eval_fairness(y_true, y_pred, A):
    mf = MetricFrame(
        metrics={
            "TPR": true_positive_rate,
            "FPR": false_positive_rate,
            "Recall": recall_score, 
            "SelectionRate": selection_rate,
            "Accuracy": accuracy_score,
        },
        y_true=y_true, y_pred=y_pred, sensitive_features=A
    )
    return {
        "by_group": mf.by_group,
        "acc": accuracy_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "dp": demographic_parity_difference(y_true, y_pred, sensitive_features=A),
        "eo": equalized_odds_difference(y_true, y_pred, sensitive_features=A),
    }

# 1) Baseline metrics (no mitigation) 
pca_knn.fit(X_train_ready, y_train)
y_base = pca_knn.predict(X_test_ready)
m_base = eval_fairness(y_test, y_base, A_test)

print("=== Baseline (tuned PCA+KNN) ===")
print(m_base["by_group"])
print(f"Accuracy: {m_base['acc']:.4f} | DP diff: {m_base['dp']:.4f} | EO diff: {m_base['eo']:.4f}")

# 2) Post-processing with DEMOGRAPHIC PARITY
post_dp = ThresholdOptimizer(
    estimator=pca_knn,
    constraints="demographic_parity",
    predict_method="predict_proba",   # KNN supports this
    grid_size=200,
    prefit=True
)
post_dp.fit(X_train_ready, y_train, sensitive_features=A_train)

y_dp = post_dp.predict(X_test_ready, sensitive_features=A_test, random_state=42)
m_dp = eval_fairness(y_test, y_dp, A_test)

print("\n=== Post-processing (Demographic Parity) ===")
print(m_dp["by_group"])
print(f"Accuracy: {m_dp['acc']:.4f} | DP diff: {m_dp['dp']:.4f} | EO diff: {m_dp['eo']:.4f}")

# 3) Post-processing with EQUALIZED ODDS
post_eod = ThresholdOptimizer(
    estimator=pca_knn,
    constraints="equalized_odds",
    predict_method="predict_proba",   # KNN supports this
    grid_size=200,
    prefit=True,                                # makes randomized post-processing reproducible
)
post_eod.fit(X_train_ready, y_train, sensitive_features=A_train)

y_eod = post_eod.predict(X_test_ready, sensitive_features=A_test, random_state=42)
m_eod = eval_fairness(y_test, y_eod, A_test)

print("\n=== Post-processing (Equalized Odds) ===")
print(m_eod["by_group"])
print(f"Accuracy: {m_eod['acc']:.4f} | DP diff: {m_eod['dp']:.4f} | EO diff: {m_eod['eo']:.4f}")


=== Baseline (tuned PCA+KNN) ===
          TPR     FPR    Recall  SelectionRate  Accuracy
Sex                                                     
0    0.666667  0.0625  0.666667       0.157895  0.894737
1    0.854167  0.1000  0.854167       0.595890  0.869863
Accuracy: 0.8750 | DP diff: 0.4380 | EO diff: 0.1875

=== Post-processing (Demographic Parity) ===
          TPR     FPR    Recall  SelectionRate  Accuracy
Sex                                                     
0    0.666667  0.0625  0.666667       0.157895  0.894737
1    0.854167  0.1000  0.854167       0.595890  0.869863
Accuracy: 0.8750 | DP diff: 0.4380 | EO diff: 0.1875

=== Post-processing (Equalized Odds) ===
          TPR     FPR    Recall  SelectionRate  Accuracy
Sex                                                     
0    0.666667  0.0625  0.666667       0.157895  0.894737
1    0.854167  0.1000  0.854167       0.595890  0.869863
Accuracy: 0.8750 | DP diff: 0.4380 | EO diff: 0.1875


### Bias Mitigation Results: PCA+KNN – Post-Processing  


#### Metrics Overview

| Model                    | Accuracy | DP diff | EO diff | Notes                                                             |
|--------------------------|:--------:|:-------:|:-------:|-------------------------------------------------------------------|
| PCA+KNN Baseline (tuned) | 0.8750   | 0.4380  | 0.1875  | Large DP gap; sizable EO gap                                      |
| PCA+KNN + PP (DP)        | 0.8750   | 0.4380  | 0.1875  | **Identical to baseline** → post-processing not applied/effective |
| PCA+KNN + PP (EO)        | 0.8750   | 0.4380  | 0.1875  | **Identical to baseline** → post-processing not applied/effective |

---

#### Interpretation
- Baseline shows **strong outcome disparity** (SelRate: **0.158** F vs **0.596** M → **DP 0.438**) and **non-trivial error-rate gap** (**EO 0.1875**).
- Both post-processing runs yield **no change**, indicating the mitigator likely didn’t use scores or its adjusted outputs weren’t used for evaluation.


---

**CorrelationRemover** will be implemented to improve fairness after DP/EOD post-processing failed to change any predictions (0% flips), leaving metrics unchanged. By removing linear correlation between features and the sensitive attribute, we reduce leakage and make group score distributions more comparable, giving PCA+KNN and also any subsequent post-processing room to adjust selection rates and error rates—all while staying.

In [11]:
from fairlearn.preprocessing import CorrelationRemover
from sklearn.metrics import recall_score  

Xtr_df = X_train_ready.copy()
Xte_df = X_test_ready.copy()
Xtr_df["__A__"] = A_train.values
Xte_df["__A__"] = A_test.values

cr = CorrelationRemover(sensitive_feature_ids=["__A__"])

Xtr_fair_arr = cr.fit_transform(Xtr_df)   # shape: (n_samples, n_features - 1)
Xte_fair_arr = cr.transform(Xte_df)

# Rebuild DataFrames with columns that exclude the sensitive column
cols_out = [c for c in Xtr_df.columns if c != "__A__"]
Xtr_fair = pd.DataFrame(Xtr_fair_arr, index=Xtr_df.index, columns=cols_out)
Xte_fair = pd.DataFrame(Xte_fair_arr, index=Xte_df.index, columns=cols_out)

# Refit your PCA+KNN
pca_knn.fit(Xtr_fair, y_train)
y_cr = pca_knn.predict(Xte_fair)
m_cr = eval_fairness(y_test, y_cr, A_test)

print("\n=== Preprocessing: CorrelationRemover + PCA+KNN ===")
print(m_cr["by_group"])
print(f"Accuracy: {m_cr['acc']:.4f} | DP diff: {m_cr['dp']:.4f} | EO diff: {m_cr['eo']:.4f}")


=== Preprocessing: CorrelationRemover + PCA+KNN ===
          TPR     FPR    Recall  SelectionRate  Accuracy
Sex                                                     
0    0.666667  0.0625  0.666667       0.157895  0.894737
1    0.822917  0.1000  0.822917       0.575342  0.849315
Accuracy: 0.8587 | DP diff: 0.4174 | EO diff: 0.1562


**Interpretation:**

- **Accuracy:** **0.8587** (↓ **1.63** pts vs baseline **0.8750**).
- **DP diff:** **0.4174** (↓ **0.0206**) — selection **Female 0.158** vs **Male 0.575** (~**3.64×** higher for males).
- **EO diff:** **0.1562** (↓ **0.0313**):
  - **TPR gap:** 0.667 vs 0.823 → **0.156** (improved from **0.188**).
  - **FPR gap:** 0.0625 vs 0.1000 → **0.0375** (**unchanged**).
- **Group accuracy:** **Female 0.895** (≈ same) vs **Male 0.849** (↓ ~2.0 pts) → **accuracy gap widens** (~0.025 → ~0.045).

**Summary:** CorrelationRemover **modestly improves DP and EO** by **lowering male sensitivity/selection**, but at the cost of **overall and male-group accuracy**. If the goal is parity without degrading the better-performing group, consider **in-processing constraints** or **group-specific thresholds** that **raise female recall** instead of reducing male performance.


---

In [12]:
from fairlearn.postprocessing import ThresholdOptimizer

# Demographic Parity on top of the CorrelationRemover
post_dp_cr = ThresholdOptimizer(
    estimator=pca_knn,
    constraints="demographic_parity",
    objective="accuracy_score",
    predict_method="predict_proba",
    grid_size=1000,
    prefit=True
)
post_dp_cr.fit(Xtr_fair, y_train, sensitive_features=A_train)  # ideally fit on a validation split
y_dp_cr = post_dp_cr.predict(Xte_fair, sensitive_features=A_test, random_state=42)
m_dp_cr = eval_fairness(y_test, y_dp_cr, A_test)

# Equalized Odds on top of CorrelationRemover
post_eod_cr = ThresholdOptimizer(
    estimator=pca_knn,
    constraints="equalized_odds",
    objective="accuracy_score",
    predict_method="predict_proba",
    grid_size=1000,
    prefit=True
)
post_eod_cr.fit(Xtr_fair, y_train, sensitive_features=A_train)  # ideally fit on a validation split
y_eod_cr = post_eod_cr.predict(Xte_fair, sensitive_features=A_test, random_state=42)
m_eod_cr = eval_fairness(y_test, y_eod_cr, A_test)


print("\n=== Post-CR (DP) ===")
print(m_dp_cr["by_group"])
print(f"Accuracy: {m_dp_cr['acc']:.4f} | DP diff: {m_dp_cr['dp']:.4f} | EO diff: {m_dp_cr['eo']:.4f}")

print("\n=== Post-CR (eOD) ===")
print(m_eod_cr["by_group"])
print(f"Accuracy: {m_eod_cr['acc']:.4f} | DP diff: {m_eod_cr['dp']:.4f} | EO diff: {m_eod_cr['eo']:.4f}")



=== Post-CR (DP) ===
          TPR     FPR    Recall  SelectionRate  Accuracy
Sex                                                     
0    0.666667  0.0625  0.666667       0.157895  0.894737
1    0.822917  0.1000  0.822917       0.575342  0.849315
Accuracy: 0.8587 | DP diff: 0.4174 | EO diff: 0.1562

=== Post-CR (eOD) ===
          TPR     FPR    Recall  SelectionRate  Accuracy
Sex                                                     
0    0.666667  0.0625  0.666667       0.157895  0.894737
1    0.822917  0.1000  0.822917       0.575342  0.849315
Accuracy: 0.8587 | DP diff: 0.4174 | EO diff: 0.1562


**Interpretation:**  
- **No effect:** Post-CR **DP** and **eOD** post-processing changed **0%** — predictions identical to the CR baseline.  
- **Disparities persist:** **DP diff 0.4174** (Female sel. **0.158** vs Male **0.575**), **EO diff 0.1562** (TPR **0.667** vs **0.823**; FPR **0.0625** vs **0.1000**).  
- **Accuracy:** **0.8587**, unchanged.


### Bias mitigation comparison (PCA+KNN)  


| Model variant                    | Accuracy | DP diff | EO diff | SelRate S=0 | SelRate S=1 | TPR S=0 | TPR S=1 | FPR S=0 | FPR S=1 | Notes                         |
|----------------------------------|:--------:|:-------:|:-------:|:-----------:|:-----------:|:-------:|:-------:|:-------:|:-------:|--------------------------------|
| Baseline (tuned PCA+KNN)         | 0.8750   | 0.4380  | 0.1875  | 0.1579      | 0.5959      | 0.6667  | 0.8542  | 0.0625  | 0.1000  | Reference                      |
| Post-processing (DP constraint)  | 0.8750   | 0.4380  | 0.1875  | 0.1579      | 0.5959      | 0.6667  | 0.8542  | 0.0625  | 0.1000  | **Flips vs baseline: 0%**      |
| Post-processing (EO constraint)  | 0.8750   | 0.4380  | 0.1875  | 0.1579      | 0.5959      | 0.6667  | 0.8542  | 0.0625  | 0.1000  | **Flips vs baseline: 0%**      |
| CorrelationRemover + PCA+KNN     | 0.8587   | 0.4174  | 0.1562  | 0.1579      | 0.5753      | 0.6667  | 0.8229  | 0.0625  | 0.1000  | New baseline after CR          |
| Post-CR (DP constraint)          | 0.8587   | 0.4174  | 0.1562  | 0.1579      | 0.5753      | 0.6667  | 0.8229  | 0.0625  | 0.1000  | **Flips vs CR baseline: 0%**   |
| Post-CR (EO constraint)          | 0.8587   | 0.4174  | 0.1562  | 0.1579      | 0.5753      | 0.6667  | 0.8229  | 0.0625  | 0.1000  | **Flips vs CR baseline: 0%**   |

**Takeaway:** Post-processing caused **no label changes** in either setting. **CorrelationRemover** modestly **reduced DP** (0.4380→0.4174) and **EO** (0.1875→0.1562) with a small **accuracy drop** (0.8750→0.8587). Male selection remains much higher than female (≈0.596→0.575 vs 0.158).

---

### Alternative Tuned & Pruned Decision Tree

In [13]:
# Alternative DT tuning: simpler trees + class balancing + cost-complexity pruning
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)
import numpy as np

# bias toward simpler trees with class_weight="balanced" 
base_dt = DecisionTreeClassifier(random_state=42, class_weight="balanced")

param_grid_simple = {
    "criterion": ["gini", "entropy"],
    "max_depth": [3, 4, 5, 6, 7],
    "min_samples_split": [5, 10, 20],
    "min_samples_leaf": [2, 4, 6],
    "min_impurity_decrease": [0.0, 1e-4, 1e-3],  # tiny regularization
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_simple = GridSearchCV(
    estimator=base_dt,
    param_grid=param_grid_simple,
    cv=cv,
    scoring="recall",    # balanced focus
    n_jobs=-1,
    verbose=0,
    refit=True
)
grid_simple.fit(X_train_ready, y_train)

print("Stage A — Best simple DT params:", grid_simple.best_params_)
print("Stage A — Best CV F1:", grid_simple.best_score_)
simple_dt = grid_simple.best_estimator_

# cost-complexity pruning
path = simple_dt.cost_complexity_pruning_path(X_train_ready, y_train)
ccp_alphas = path.ccp_alphas

unique_alphas = np.unique(np.round(ccp_alphas, 6))
candidate_alphas = np.linspace(unique_alphas.min(), unique_alphas.max(), num=min(20, len(unique_alphas)))
candidate_alphas = np.unique(np.concatenate([candidate_alphas, [0.0]]))  

cv_scores = []
for alpha in candidate_alphas:
    dt_alpha = DecisionTreeClassifier(
        random_state=42,
        class_weight="balanced",
        criterion=simple_dt.criterion,
        max_depth=simple_dt.max_depth,
        min_samples_split=simple_dt.min_samples_split,
        min_samples_leaf=simple_dt.min_samples_leaf,
        min_impurity_decrease=simple_dt.min_impurity_decrease,
        ccp_alpha=alpha
    )
    f1_cv = cross_val_score(dt_alpha, X_train_ready, y_train, cv=cv, scoring="recall", n_jobs=-1).mean()
    cv_scores.append((alpha, f1_cv))

best_alpha, best_cv_f1 = sorted(cv_scores, key=lambda x: x[1], reverse=True)[0]
print(f"Stage B — Best ccp_alpha: {best_alpha:.6f} | CV F1: {best_cv_f1:.4f}")

best_dt = DecisionTreeClassifier(
    random_state=42,
    class_weight="balanced",
    criterion=simple_dt.criterion,
    max_depth=simple_dt.max_depth,
    min_samples_split=simple_dt.min_samples_split,
    min_samples_leaf=simple_dt.min_samples_leaf,
    min_impurity_decrease=simple_dt.min_impurity_decrease,
    ccp_alpha=best_alpha
).fit(X_train_ready, y_train)

# Evaluate on test set 
y_pred_dt = best_dt.predict(X_test_ready)
y_prob_dt = best_dt.predict_proba(X_test_ready)[:, 1]  

evaluate_model(y_test, y_pred_dt, "Alternative Tuned & Pruned Decision Tree")

Stage A — Best simple DT params: {'criterion': 'entropy', 'max_depth': 6, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 4, 'min_samples_split': 5}
Stage A — Best CV F1: 0.8800000000000001
Stage B — Best ccp_alpha: 0.000000 | CV F1: 0.8800
=== Alternative Tuned & Pruned Decision Tree Evaluation ===
Accuracy : 0.8206521739130435
Precision: 0.896551724137931
Recall   : 0.7647058823529411
F1 Score : 0.8253968253968254

Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.89      0.82        82
           1       0.90      0.76      0.83       102

    accuracy                           0.82       184
   macro avg       0.82      0.83      0.82       184
weighted avg       0.83      0.82      0.82       184

Confusion Matrix:
 [[73  9]
 [24 78]]




### Bias Mitigation DT: Inprocessing - Exponentiated Gradient Reduction

In [14]:
# In-processing mitigation for tuned Decision Tree
from sklearn.base import clone
from fairlearn.reductions import ExponentiatedGradient, EqualizedOdds, DemographicParity
from fairlearn.metrics import (
    MetricFrame, true_positive_rate, false_positive_rate, selection_rate,
    demographic_parity_difference, equalized_odds_difference
)
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

# 0) Baseline: tuned DT without mitigation (for comparison)
y_pred_dt_base = best_dt.predict(X_test_ready)
m_base = eval_fairness(y_test, y_pred_dt_base, A_test)
print("=== Baseline (Tuned DT) ===")
print(m_base["by_group"])
print(f"Accuracy: {m_base['acc']:.4f} | DP diff: {m_base['dp']:.4f} | EO diff: {m_base['eo']:.4f}")

# 1) Exponentiated Gradient with Equalized Odds
eg_eo = ExponentiatedGradient(
    estimator=clone(best_dt),        
    constraints=EqualizedOdds(),
    eps=0.01,                         
    max_iter=50
)
eg_eo.fit(X_train_ready, y_train, sensitive_features=A_train)
y_pred_eo = eg_eo.predict(X_test_ready)
m_eo = eval_fairness(y_test, y_pred_eo, A_test)
print("\n=== In-processing: EG (Equalized Odds) ===")
print(m_eo["by_group"])
print(f"Accuracy: {m_eo['acc']:.4f} | DP diff: {m_eo['dp']:.4f} | EO diff: {m_eo['eo']:.4f}")

# 2) Exponentiated Gradient with Demographic Parity
eg_dp = ExponentiatedGradient(
    estimator=clone(best_dt),
    constraints=DemographicParity(),
    eps=0.01,
    max_iter=50
)
eg_dp.fit(X_train_ready, y_train, sensitive_features=A_train)
y_pred_dp = eg_dp.predict(X_test_ready)
m_dp = eval_fairness(y_test, y_pred_dp, A_test)
print("\n=== In-processing: EG (Demographic Parity) ===")
print(m_dp["by_group"])
print(f"Accuracy: {m_dp['acc']:.4f} | DP diff: {m_dp['dp']:.4f} | EO diff: {m_dp['eo']:.4f}")

# 3) Summary table
summary_dt = pd.DataFrame([
    {"model":"DT Baseline (tuned)", "accuracy":m_base["acc"], "dp_diff":m_base["dp"], "eo_diff":m_base["eo"]},
    {"model":"DT + EG (EO)",        "accuracy":m_eo["acc"],   "dp_diff":m_eo["dp"],   "eo_diff":m_eo["eo"]},
    {"model":"DT + EG (DP)",        "accuracy":m_dp["acc"],   "dp_diff":m_dp["dp"],   "eo_diff":m_dp["eo"]},
]).round(4)
print("\n=== Decision Tree: Baseline vs In-processing (EG) ===")
summary_dt

=== Baseline (Tuned DT) ===
          TPR    FPR    Recall  SelectionRate  Accuracy
Sex                                                    
0    0.666667  0.125  0.666667       0.210526  0.842105
1    0.770833  0.100  0.770833       0.541096  0.815068
Accuracy: 0.8207 | DP diff: 0.3306 | EO diff: 0.1042

=== In-processing: EG (Equalized Odds) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.500000  0.09375  0.500000       0.157895  0.842105
1    0.822917  0.18000  0.822917       0.602740  0.821918
Accuracy: 0.8261 | DP diff: 0.4448 | EO diff: 0.3229

=== In-processing: EG (Demographic Parity) ===
          TPR   FPR    Recall  SelectionRate  Accuracy
Sex                                                   
0    0.833333  0.25  0.833333       0.342105  0.763158
1    0.812500  0.26  0.812500       0.623288  0.787671
Accuracy: 0.7826 | DP diff: 0.2812 | EO diff: 0.0208

=== Decision Tree: Baseline vs In-processing

,model,accuracy,dp_diff,eo_diff
0,DT Baseline (tuned),0.8207,0.3306,0.1042
1,DT + EG (EO),0.8261,0.4448,0.3229
2,DT + EG (DP),0.7826,0.2812,0.0208


### Bias Mitigation Results: Decision Tree – In-Processing  

#### Metrics Overview  

| Model                   | Accuracy | DP diff | EO diff | Notes                                                                 |
|--------------------------|:--------:|:-------:|:-------:|-----------------------------------------------------------------------|
| **DT Baseline (tuned)** | 0.8207   | 0.3306  | 0.1042  | Strong DP gap; moderate EO gap.                                       |
| **DT + EG (EO)**        | 0.8261   | 0.4448  | 0.3229  | Higher accuracy, but **fairness worsened** (larger DP & EO gaps).     |
| **DT + EG (DP)**        | 0.7826   | 0.2812  | 0.0208  | Accuracy dropped, but **fairness improved** (smaller DP & especially EO gaps). |

---

#### Interpretation  

- **Baseline (tuned DT):**  
  - Accuracy ≈ **82%**, but large **selection rate gap** (0.21 vs. 0.54, DP diff = 0.33).  
  - Moderate EO gap (**0.10**) — indicating some imbalance in error rates across sexes.  

- **Exponentiated Gradient (Equalized Odds):**  
  - Slight **accuracy gain** (82.6%), but fairness degraded.  
  - DP diff **increased** to 0.44, EO diff **tripled** to 0.32.  
  - → Trade-off skewed toward accuracy at the expense of equity.  

- **Exponentiated Gradient (Demographic Parity):**  
  - Accuracy decreased (**78%**), but fairness improved.  
  - DP diff reduced to **0.28** and EO diff nearly eliminated (**0.02**).  
  - → Strong fairness gains, especially in equalizing true/false positive rates, though with some cost in predictive performance.  

- **Conclusion:**  
  - **EG (EO)** may not be suitable here, as it amplifies disparities.  
  - **EG (DP)** shows a better fairness–accuracy trade-off, reducing both DP and EO gaps, albeit with a slight drop in accuracy.  

---

#### Bias Mitigation DT: In-processing: GridSearch Reduction

In [15]:
from sklearn.base import clone
from fairlearn.reductions import GridSearch, EqualizedOdds, DemographicParity

# 1) GridSearch with Equalized Odds
gs_eo = GridSearch(
    estimator=clone(best_dt),              # unfitted clone of tuned DT
    constraints=EqualizedOdds(),            # EO constraint
    selection_rule="tradeoff_optimization", 
    constraint_weight=0.5,                  
    grid_size=15,                           
)
gs_eo.fit(X_train_ready, y_train, sensitive_features=A_train)
y_pred_gs_eo = gs_eo.predict(X_test_ready)
m_gs_eo = eval_fairness(y_test, y_pred_gs_eo, A_test)
print("\n=== In-processing: GridSearch (Equalized Odds) ===")
print(m_gs_eo["by_group"])
print(f"Accuracy: {m_gs_eo['acc']:.4f} | DP diff: {m_gs_eo['dp']:.4f} | EO diff: {m_gs_eo['eo']:.4f}")

# 2) GridSearch with Demographic Parity
gs_dp = GridSearch(
    estimator=clone(best_dt),
    constraints=DemographicParity(),
    selection_rule="tradeoff_optimization",
    constraint_weight=0.5,
    grid_size=15,
)
gs_dp.fit(X_train_ready, y_train, sensitive_features=A_train)
y_pred_gs_dp = gs_dp.predict(X_test_ready)
m_gs_dp = eval_fairness(y_test, y_pred_gs_dp, A_test)
print("\n=== In-processing: GridSearch (Demographic Parity) ===")
print(m_gs_dp["by_group"])
print(f"Accuracy: {m_gs_dp['acc']:.4f} | DP diff: {m_gs_dp['dp']:.4f} | EO diff: {m_gs_dp['eo']:.4f}")

# 3) Compare with your existing runs
summary_dt = pd.concat([
    summary_dt,  
    pd.DataFrame([
        {"model":"DT + GS (EO)", "accuracy":m_gs_eo["acc"], "dp_diff":m_gs_eo["dp"], "eo_diff":m_gs_eo["eo"]},
        {"model":"DT + GS (DP)", "accuracy":m_gs_dp["acc"], "dp_diff":m_gs_dp["dp"], "eo_diff":m_gs_dp["eo"]},
    ]).round(4)
], ignore_index=True)
print("\n=== Decision Tree: Baseline vs EG vs GS ===")
summary_dt


=== In-processing: GridSearch (Equalized Odds) ===
          TPR    FPR    Recall  SelectionRate  Accuracy
Sex                                                    
0    0.666667  0.125  0.666667       0.210526  0.842105
1    0.770833  0.100  0.770833       0.541096  0.815068
Accuracy: 0.8207 | DP diff: 0.3306 | EO diff: 0.1042

=== In-processing: GridSearch (Demographic Parity) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    1.000000  0.21875  1.000000       0.342105  0.815789
1    0.802083  0.26000  0.802083       0.616438  0.780822
Accuracy: 0.7880 | DP diff: 0.2743 | EO diff: 0.1979

=== Decision Tree: Baseline vs EG vs GS ===


,model,accuracy,dp_diff,eo_diff
0,DT Baseline (tuned),0.8207,0.3306,0.1042
1,DT + EG (EO),0.8261,0.4448,0.3229
2,DT + EG (DP),0.7826,0.2812,0.0208
3,DT + GS (EO),0.8207,0.3306,0.1042
4,DT + GS (DP),0.7880,0.2743,0.1979


### Decision Tree — In-Processing: EG vs. GridSearch (EO & DP)  

#### Summary of results
| Model                   | Accuracy | DP diff | EO diff | Notes |
|--------------------------|:--------:|:-------:|:-------:|-------|
| **DT Baseline (tuned)** | 0.8207   | 0.3306  | 0.1042  | Reference |
| **DT + EG (EO)**        | 0.8261   | 0.4448  | 0.3229  | Higher accuracy, but fairness worsened |
| **DT + EG (DP)**        | 0.7826   | 0.2812  | 0.0208  | Lower accuracy, but fairness improved |
| **DT + GS (EO)**        | 0.8207   | 0.3306  | 0.1042  | **Identical to baseline** (no effect) |
| **DT + GS (DP)**        | 0.7880   | 0.2743  | 0.1979  | Lower accuracy, DP gap smaller but EO gap larger |

---

#### Interpretation
- **Baseline (tuned DT):**  
  - Accuracy ≈ **82%**, with a substantial **DP gap (0.33)** and moderate **EO gap (0.10)**.  

- **Exponentiated Gradient (EG):**  
  - **EO constraint:** Increased accuracy slightly, but fairness deteriorated (both DP and EO gaps worsened).  
  - **DP constraint:** Accuracy dropped, but fairness improved significantly — especially EO gap (down to 0.02).  

- **GridSearch (GS):**  
  - **EO constraint:** Produced results **identical to the baseline** (no impact).  
  - **DP constraint:** Reduced DP gap somewhat, but EO gap worsened (≈ 0.20). Accuracy also dropped.  

---

**Conclusion:**  
- **EG (DP)** gave the best fairness–accuracy trade-off, reducing disparities while accepting a modest performance loss.  
- **EG (EO)** and **GS (DP)** highlight that not all fairness constraints move the model in the desired direction — they can worsen other gaps.  
- **GS (EO)** had **no effect**, showing the constraint was not binding in this setup.  

---  

#### Bias Mitigation DT: Post-processing: Threshold Optimizer 

In [16]:
from fairlearn.postprocessing import ThresholdOptimizer

#Baseline for mitigation: fixed tuned DT
best_dt.fit(X_train_ready, y_train)
y_base = best_dt.predict(X_test_ready)
m_base = eval_fairness(y_test, y_base, A_test)
print("=== Baseline (tuned DT) ===")
print(m_base["by_group"])
print(f"Accuracy: {m_base['acc']:.4f} | DP diff: {m_base['dp']:.4f} | EO diff: {m_base['eo']:.4f}")

#Post-processing: Equalized Odds
post_eo = ThresholdOptimizer(
    estimator=best_dt,
    constraints="equalized_odds",
    predict_method="predict_proba",   
    grid_size=200,
    flip=True
)
post_eo.fit(X_train_ready, y_train, sensitive_features=A_train)
y_eo = post_eo.predict(X_test_ready, sensitive_features=A_test, random_state=42)
m_eo = eval_fairness(y_test, y_eo, A_test)
print("\n=== Post-processing (Equalized Odds) ===")
print(m_eo["by_group"])
print(f"Accuracy: {m_eo['acc']:.4f} | DP diff: {m_eo['dp']:.4f} | EO diff: {m_eo['eo']:.4f}")

# Post-processing: Demographic Parity
post_dp = ThresholdOptimizer(
    estimator=best_dt,
    constraints="demographic_parity",
    predict_method="predict_proba",
    grid_size=200,
    flip=True
)
post_dp.fit(X_train_ready, y_train, sensitive_features=A_train)
y_dp = post_dp.predict(X_test_ready, sensitive_features=A_test, random_state=42)
m_dp = eval_fairness(y_test, y_dp, A_test)
print("\n=== Post-processing (Demographic Parity) ===")
print(m_dp["by_group"])
print(f"Accuracy: {m_dp['acc']:.4f} | DP diff: {m_dp['dp']:.4f} | EO diff: {m_dp['eo']:.4f}")

# create summary table 
summary = pd.DataFrame([
    {"model":"DT Baseline (tuned)", "accuracy":m_base["acc"], "dp_diff":m_base["dp"], "eo_diff":m_base["eo"]},
    {"model":"DT + Post (EO)",      "accuracy":m_eo["acc"],   "dp_diff":m_eo["dp"],   "eo_diff":m_eo["eo"]},
    {"model":"DT + Post (DP)",      "accuracy":m_dp["acc"],   "dp_diff":m_dp["dp"],   "eo_diff":m_dp["eo"]},
]).round(4)
print("\n=== Decision Tree: Baseline vs Post-processing ===")
summary

=== Baseline (tuned DT) ===
          TPR    FPR    Recall  SelectionRate  Accuracy
Sex                                                    
0    0.666667  0.125  0.666667       0.210526  0.842105
1    0.770833  0.100  0.770833       0.541096  0.815068
Accuracy: 0.8207 | DP diff: 0.3306 | EO diff: 0.1042

=== Post-processing (Equalized Odds) ===
          TPR     FPR    Recall  SelectionRate  Accuracy
Sex                                                     
0    0.666667  0.1875  0.666667       0.263158  0.789474
1    0.833333  0.1400  0.833333       0.595890  0.842466
Accuracy: 0.8315 | DP diff: 0.3327 | EO diff: 0.1667

=== Post-processing (Demographic Parity) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.500000  0.09375  0.500000       0.157895  0.842105
1    0.833333  0.14000  0.833333       0.595890  0.842466
Accuracy: 0.8424 | DP diff: 0.4380 | EO diff: 0.3333

=== Decision Tree: Baseline vs Post-proc

,model,accuracy,dp_diff,eo_diff
0,DT Baseline (tuned),0.8207,0.3306,0.1042
1,DT + Post (EO),0.8315,0.3327,0.1667
2,DT + Post (DP),0.8424,0.4380,0.3333


### Decision Tree — Post- vs In-Processing  

#### Combined Results

| Model / Method          | Accuracy | DP diff | EO diff | Notes (vs. baseline 0.8207 / 0.3306 / 0.1042) |
|--------------------------|:--------:|:-------:|:-------:|-----------------------------------------------|
| **Baseline (Tuned DT)** | 0.8207   | 0.3306  | 0.1042  | Reference                                      |
| **Post (EO)**           | 0.8315   | 0.3327  | 0.1667  | Accuracy ↑ (+1.1 pts); DP ≈ baseline; **EO worsened** |
| **Post (DP)**           | 0.8424   | 0.4380  | 0.3333  | Accuracy ↑ (+2.2 pts); **DP worsened**; **EO worsened significantly** |
| **EG (EO)**             | 0.8261   | 0.4448  | 0.3229  | Accuracy ↑; **fairness worsened** (DP & EO higher) |
| **EG (DP)**             | 0.7826   | 0.2812  | 0.0208  | Accuracy ↓; **fairness improved** (EO nearly eliminated, DP smaller) |
| **GS (EO)**             | 0.8207   | 0.3306  | 0.1042  | **Identical to baseline** (no effect) |
| **GS (DP)**             | 0.7880   | 0.2743  | 0.1979  | Accuracy ↓; DP improved slightly, EO worsened |

---

#### Interpretation
- **Baseline (tuned DT):** Balanced accuracy (~82%) but a **large DP gap** (0.33) and moderate **EO gap** (0.10).  
- **Post-processing (EO):** Accuracy improved slightly, but **EO disparity increased** (0.17). No DP relief.  
- **Post-processing (DP):** Highest accuracy, but **fairness strongly worsened** — both DP and EO gaps grew.  
- **Exponentiated Gradient (EG):**  
  - **EO constraint:** Slight accuracy gain, but fairness deteriorated.  
  - **DP constraint:** Lower accuracy, but **fairness gains**, especially EO (0.02).  
- **GridSearch (GS):**  
  - **EO constraint:** No effect (baseline results repeated).  
  - **DP constraint:** Small DP improvement, but EO worsened and accuracy dropped.  

---

**Takeaway:**  
- **Post-processing did not help fairness** in this setup — in fact, both EO and DP disparities worsened while accuracy increased.  
- **EG with DP constraint** provided the **most meaningful fairness improvement** (especially EO gap reduction) at a modest accuracy cost.  
- **GS (EO)** had no effect, while **GS (DP)** gave only marginal DP gains but worsened EO.  
- Overall, **only EG (DP)** demonstrated a genuine fairness–accuracy trade-off; other methods either worsened disparities or had no effect.  

---

### Ensemble Model - Random Forest (RF)

In [17]:
from sklearn.ensemble import RandomForestClassifier

# Initialize Random Forest
rf = RandomForestClassifier(random_state=42)

# Train the model
rf.fit(X_train_ready, y_train)

# Predict on test set
y_pred_rf = rf.predict(X_test_ready)
y_prob_rf = rf.predict_proba(X_test_ready)[:, 1]  
evaluate_model(y_test, y_pred_rf, "Random Forest")

=== Random Forest Evaluation ===
Accuracy : 0.8315217391304348
Precision: 0.8585858585858586
Recall   : 0.8333333333333334
F1 Score : 0.845771144278607

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.83      0.81        82
           1       0.86      0.83      0.85       102

    accuracy                           0.83       184
   macro avg       0.83      0.83      0.83       184
weighted avg       0.83      0.83      0.83       184

Confusion Matrix:
 [[68 14]
 [17 85]]




### Bias Mitgation RF: In-processing: Exponentiated Gradient 

In [18]:
# 0) Baseline Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_ready, y_train)

y_pred_rf_base = rf.predict(X_test_ready)
m_rf_base = eval_fairness(y_test, y_pred_rf_base, A_test)

print("=== Baseline (Random Forest) ===")
print(m_rf_base["by_group"])
print(f"Accuracy: {m_rf_base['acc']:.4f} | DP diff: {m_rf_base['dp']:.4f} | EO diff: {m_rf_base['eo']:.4f}")

#1) EG with Equalized Odds
eg_eo_rf = ExponentiatedGradient(
    estimator=clone(rf),
    constraints=EqualizedOdds(),
    eps=0.01,
    max_iter=50
)
eg_eo_rf.fit(X_train_ready, y_train, sensitive_features=A_train)
y_pred_rf_eo = eg_eo_rf.predict(X_test_ready, random_state=42)
m_rf_eo = eval_fairness(y_test, y_pred_rf_eo, A_test)

print("\n=== In-processing RF: EG (Equalized Odds) ===")
print(m_rf_eo["by_group"])
print(f"Accuracy: {m_rf_eo['acc']:.4f} | DP diff: {m_rf_eo['dp']:.4f} | EO diff: {m_rf_eo['eo']:.4f}")

# 2) EG with Demographic Parity 
eg_dp_rf = ExponentiatedGradient(
    estimator=clone(rf),
    constraints=DemographicParity(),
    eps=0.01,
    max_iter=50
)
eg_dp_rf.fit(X_train_ready, y_train, sensitive_features=A_train)
y_pred_rf_dp = eg_dp_rf.predict(X_test_ready, random_state=42)
m_rf_dp = eval_fairness(y_test, y_pred_rf_dp, A_test)

print("\n=== In-processing RF: EG (Demographic Parity) ===")
print(m_rf_dp["by_group"])
print(f"Accuracy: {m_rf_dp['acc']:.4f} | DP diff: {m_rf_dp['dp']:.4f} | EO diff: {m_rf_dp['eo']:.4f}")

# 3) Summary Table 
summary_rf = pd.DataFrame([
    {"model":"RF Baseline",      "accuracy":m_rf_base["acc"], "dp_diff":m_rf_base["dp"], "eo_diff":m_rf_base["eo"]},
    {"model":"RF + EG (EO)",     "accuracy":m_rf_eo["acc"],   "dp_diff":m_rf_eo["dp"],   "eo_diff":m_rf_eo["eo"]},
    {"model":"RF + EG (DP)",     "accuracy":m_rf_dp["acc"],   "dp_diff":m_rf_dp["dp"],   "eo_diff":m_rf_dp["eo"]},
]).round(4)

print("\n=== Random Forest: Baseline vs In-processing (EG) ===")
print(summary_rf)

=== Baseline (Random Forest) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.833333  0.15625  0.833333       0.263158  0.842105
1    0.833333  0.18000  0.833333       0.609589  0.828767
Accuracy: 0.8315 | DP diff: 0.3464 | EO diff: 0.0237

=== In-processing RF: EG (Equalized Odds) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.833333  0.15625  0.833333       0.263158  0.842105
1    0.833333  0.18000  0.833333       0.609589  0.828767
Accuracy: 0.8315 | DP diff: 0.3464 | EO diff: 0.0237

=== In-processing RF: EG (Demographic Parity) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.833333  0.15625  0.833333       0.263158  0.842105
1    0.833333  0.18000  0.833333       0.609589  0.828767
Accuracy: 0.8315 | DP diff: 0.3464 | EO diff: 0.0237

=== Random Fo

### Random Forest Bias Mitigation Results  

### Summary

| Model            | Accuracy | DP diff | EO diff | Interpretation                                  |
|------------------|:--------:|:-------:|:-------:|-------------------------------------------------|
| **RF Baseline**  | 0.8315   | 0.3464  | 0.0237  | Good accuracy; **moderate DP gap**; EO already low. |
| **RF + EG (EO)** | 0.8315   | 0.3464  | 0.0237  | **No change** vs baseline → EO constraint had no effect. |
| **RF + EG (DP)** | 0.8315   | 0.3464  | 0.0237  | **No change** vs baseline → DP constraint had no effect. |

### Key Points
- **Selection rates:** Female **0.263** vs Male **0.610** → **DP 0.346** (~**2.3×** higher for males).
- **Error rates:** **TPR** equal (0.833 vs 0.833); **FPR** close (0.156 vs 0.180) → **EO 0.024** (very small).
- **EG (EO/DP)** produced **0% movement** — constraints likely not binding or optimization stayed at the baseline frontier point.

---

### Bias Mitigation: RF: In-processing: Grid Search

In [19]:
from sklearn.base import clone
from fairlearn.reductions import GridSearch, EqualizedOdds, DemographicParity

weights = [0.0, 0.25, 0.5, 0.75, 1.0]   # 0.0 = accuracy-first, 1.0 = fairness-first
grid = 50                               

rows = []

#Equalized Odds sweep
for w in weights:
    gs_eo_rf = GridSearch(
        estimator=clone(rf),                 
        constraints=EqualizedOdds(),
        selection_rule="tradeoff_optimization",
        constraint_weight=w,
        grid_size=grid
    )
    gs_eo_rf.fit(X_train_ready, y_train, sensitive_features=A_train)
    # Some versions accept random_state in predict; if yours doesn't, seed numpy before predicting
    try:
        y_hat = gs_eo_rf.predict(X_test_ready, random_state=42)
    except TypeError:
        import numpy as np, random
        np.random.seed(42); random.seed(42)
        y_hat = gs_eo_rf.predict(X_test_ready)
    m = eval_fairness(y_test, y_hat, A_test)
    rows.append({"method":"RF + GS (EO)", "weight": w, "acc": m["acc"], "dp": m["dp"], "eo": m["eo"]})

# Demographic Parity sweep
for w in weights:
    gs_dp_rf = GridSearch(
        estimator=clone(rf),
        constraints=DemographicParity(),
        selection_rule="tradeoff_optimization",
        constraint_weight=w,
        grid_size=grid
    )
    gs_dp_rf.fit(X_train_ready, y_train, sensitive_features=A_train)
    try:
        y_hat = gs_dp_rf.predict(X_test_ready, random_state=42)
    except TypeError:
        import numpy as np, random
        np.random.seed(42); random.seed(42)
        y_hat = gs_dp_rf.predict(X_test_ready)
    m = eval_fairness(y_test, y_hat, A_test)
    rows.append({"method":"RF + GS (DP)", "weight": w, "acc": m["acc"], "dp": m["dp"], "eo": m["eo"]})

df_gs = pd.DataFrame(rows).sort_values(["method","weight"])
print(df_gs)

         method  weight       acc        dp      eo
5  RF + GS (DP)    0.00  0.864130  0.451694  0.1875
6  RF + GS (DP)    0.25  0.864130  0.451694  0.1875
7  RF + GS (DP)    0.50  0.864130  0.451694  0.1875
8  RF + GS (DP)    0.75  0.864130  0.451694  0.1875
9  RF + GS (DP)    1.00  0.864130  0.451694  0.1875
0  RF + GS (EO)    0.00  0.858696  0.425379  0.0975
1  RF + GS (EO)    0.25  0.858696  0.425379  0.0975
2  RF + GS (EO)    0.50  0.858696  0.425379  0.0975
3  RF + GS (EO)    0.75  0.858696  0.425379  0.0975
4  RF + GS (EO)    1.00  0.858696  0.425379  0.0975


### Interpretation (RF + GridSearch)  

- **No movement across weights:** For both **DP** and **EO** constraints, varying the weight **0 → 1** yields the **same metrics** each time.  
- **Compared to baseline:**  

| Method   | Acc    | ΔAcc  | DP     | ΔDP   | EO     | ΔEO   | Read |
|----------|:------:|:-----:|:------:|:-----:|:------:|:-----:|------|
| GS (DP)  | 0.8641 | −1.6  | 0.4517 | +0.044 | 0.1875 | +0.104 | Accuracy ↓; **DP & EO worsen** |
| GS (EO)  | 0.8587 | −2.2  | 0.4254 | +0.017 | 0.0975 | +0.014 | Accuracy ↓; **EO slightly better than GS(DP)**, but still ↑ vs baseline |

---

### Takeaway
GridSearch **locked onto single frontier points** regardless of weight.  
- Both GS(DP) and GS(EO) **reduced accuracy** compared to the baseline RF.  
- **Fairness worsened in all cases**: DP gaps ↑ and EO gaps ↑.  
- **GS(EO)** is the less harmful option (smaller increases), but still offers **no improvement over the baseline**.  

**Conclusion:** In this setup, RF + GridSearch does **not provide a fairness–utility gain**; it simply trades off accuracy for worse fairness.

---

In [20]:
# Inspect how many distinct models GridSearch actually produced
len(gs_eo_rf.predictors_), len(gs_dp_rf.predictors_)

# See the spread across the frontier (test metrics for each predictor)
def eval_frontier(gs, X, y, A):
    rows=[]
    for i, clf in enumerate(gs.predictors_):
        yhat = clf.predict(X)
        m = eval_fairness(y, yhat, A)
        rows.append({"i": i, "acc": m["acc"], "dp": m["dp"], "eo": m["eo"]})
    return pd.DataFrame(rows)

print(eval_frontier(gs_eo_rf, X_test_ready, y_test, A_test))
print(eval_frontier(gs_dp_rf, X_test_ready, y_test, A_test))


     i       acc        dp        eo
0    0  0.836957  0.575342  0.812500
1    1  0.820652  0.554795  0.781250
2    2  0.847826  0.589041  0.833333
3    3  0.815217  0.561644  0.781250
4    4  0.695652  1.000000  1.000000
5    5  0.831522  0.568493  0.802083
6    6  0.847826  0.589041  0.833333
7    7  0.836957  0.575342  0.812500
8    8  0.684783  0.284066  0.800000
9    9  0.695652  1.000000  1.000000
10  10  0.695652  1.000000  1.000000
11  11  0.826087  0.561644  0.791667
12  12  0.820652  0.554795  0.781250
13  13  0.842391  0.582192  0.822917
14  14  0.706522  0.264600  0.808750
15  15  0.701087  0.285148  0.828750
16  16  0.684783  0.736842  0.812500
17  17  0.690217  0.763158  0.843750
18  18  0.690217  0.763158  0.843750
19  19  0.858696  0.425379  0.097500
20  20  0.875000  0.471161  0.140000
21  21  0.826087  0.378515  0.086250
22  22  0.711957  0.349315  0.800000
23  23  0.711957  0.349315  0.800000
24  24  0.706522  0.356164  0.800000
25  25  0.690217  0.763158  0.843750
2

### RF GridSearch frontiers  
**Baseline:** Acc **0.8804**, DP **0.4081**, EO **0.0833**

**What the tables show:** Each index `i` is a GridSearch candidate. Many points are **degenerate** (e.g., `i=0–17` in table 1 and `i=0–12` in table 2: DP/EO = 1.0 or very low accuracy).  
Useful frontier candidates emerge around **`i=25/30`** (EO ≈ **0.0238**, DP ≈ **0.3464**, Acc ≈ **0.8315**) and other mid–high accuracy models.

---

#### Notable candidates
| Candidate | Acc    | DP      | EO      | How it compares to baseline |
|-----------|:------:|:-------:|:-------:|------------------------------|
| **i=25/30** (tbl1/tbl2) | 0.8315 | 0.3464 | **0.0238** | **Baseline-like**: EO-minimal, slightly lower Acc |
| **i=24/39** (tbl1/tbl2) | 0.8424 | 0.3796 | 0.0550 | Acc ↑ (+1.1 pp), EO ↑ (but still small), DP ↑ |
| **i=29** (table 1)      | 0.8533 | 0.4048 | 0.0775 | Acc ↑ (+2.2 pp), DP & EO ↑ (accuracy-leaning) |
| **i=14** (table 2)      | 0.8750 | 0.3991 | 0.0313 | **Highest accuracy** (+4.3 pp), EO small, DP ↑ |

---

#### Interpretation
- **No Pareto-better solution:** No candidate simultaneously improves **DP**, **EO**, and **accuracy** vs the baseline.  
- **Baseline-like (i=25/30):** Best for **minimizing EO** (≈0.024), but with lower accuracy than the true baseline.  
- **Accuracy-focused (i=14, i=29):** Both increase accuracy but at the cost of higher DP and EO.  
- **Balanced step (i=24/39):** Slight accuracy gain with only modest fairness deterioration.  
- Candidates with **lower DP** than baseline exist only at **poor accuracy levels** (not clinically acceptable).  

---

**Summary:**  
The RF frontier primarily trades **accuracy gains for fairness losses**.  
- If the clinical goal is **error-rate parity (low EO)** → stay near **i=25/30**.  
- If **accuracy** is prioritized, **i=14** is strongest (Acc ≈ 0.875, EO small but ↑, DP worse).  
- If you want a **mild trade-off**, **i=24/39** offers slightly higher accuracy with EO/DP still moderate.  

---

In [21]:
# Show results for the specific frontier models
# for both RF GridSearch runs (EO- and DP-constrained).

import pandas as pd

indices = [25,14]

def eval_selected(gs, label):
    rows = []
    n = len(gs.predictors_)
    print(f"\n=== {label}: {n} frontier candidates ===")
    for i in indices:
        if i >= n:
            print(f"[{label}] Skipping i={i} (only {n} candidates).")
            continue
        clf = gs.predictors_[i]
        y_hat = clf.predict(X_test_ready)
        m = eval_fairness(y_test, y_hat, A_test)
        rows.append({"i": i, "accuracy": m["acc"], "dp_diff": m["dp"], "eo_diff": m["eo"]})

        # Per-group breakdown for this model
        print(f"\n[{label}] i={i}")
        print(m["by_group"])
        print(f"Accuracy: {m['acc']:.4f} | DP diff: {m['dp']:.4f} | EO diff: {m['eo']:.4f}")

    if rows:
        df = pd.DataFrame(rows).sort_values("i").round(4)
        print(f"\n--- Summary ({label}) ---")
        print(df)

# Evaluate selected indices for both EO and DP GridSearch objects
eval_selected(gs_eo_rf, "RF + GS (EO)")
eval_selected(gs_dp_rf, "RF + GS (DP)")


=== RF + GS (EO): 50 frontier candidates ===

[RF + GS (EO)] i=25
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.666667  0.15625  0.666667       0.236842  0.815789
1    1.000000  1.00000  1.000000       1.000000  0.657534
Accuracy: 0.6902 | DP diff: 0.7632 | EO diff: 0.8438

[RF + GS (EO)] i=14
       TPR      FPR  Recall  SelectionRate  Accuracy
Sex                                                 
0    0.500  0.96875   0.500       0.894737  0.105263
1    0.875  0.16000   0.875       0.630137  0.863014
Accuracy: 0.7065 | DP diff: 0.2646 | EO diff: 0.8087

--- Summary (RF + GS (EO)) ---
    i  accuracy  dp_diff  eo_diff
1  14    0.7065   0.2646   0.8088
0  25    0.6902   0.7632   0.8438

=== RF + GS (DP): 50 frontier candidates ===

[RF + GS (DP)] i=25
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.833333  0.15625  0.833333       0.2631

### RF + GridSearch — Interpretation  

#### GS (Equalized Odds constraint)  
| Candidate | Accuracy | DP diff | EO diff | Quick read |
|-----------|:--------:|:-------:|:-------:|------------|
| **i=25**  | 0.6902   | 0.7632  | 0.8438  | Reject: very poor accuracy, extreme DP/EO gaps |
| i=14      | 0.7065   | 0.2646  | 0.8088  | Reject: slightly better DP, but EO catastrophic (F FPR ≈ 0.969) |

**Summary:** Under EO constraint, **no viable candidate** emerges. Both i=14 and i=25 are dominated by **huge error-rate disparities** and **low accuracy**.  

---

#### GS (Demographic Parity constraint)  
| Candidate | Accuracy | DP diff | EO diff | Quick read |
|-----------|:--------:|:-------:|:-------:|------------|
| **i=14**  | **0.8750** | 0.3991 | **0.0312** | Best trade-off: **highest accuracy**, strong EO improvement |
| i=25      | 0.8315   | **0.3464** | 0.0237 | Better DP & EO, but accuracy lower |

**Why i=14 (DP) stands out:**  
- **Accuracy ↑** to 0.8750 (best across candidates).  
- **EO ↓** to 0.0312 (near parity, driven by balanced TPR/FPR: 0.833 vs 0.865, 0.094 vs 0.120).  
- **DP gap** remains large (0.211 vs 0.610 → DP ≈ 0.399), only slightly worse than baseline.  

---

### Takeaways  
- **EO constraint (GS-EO):** No usable points — EO exploded and accuracy collapsed.  
- **DP constraint (GS-DP):**  
  - **i=14** offers the **best balance**: highest accuracy + very low EO, though DP gap persists.  
  - **i=25** gives slightly lower DP (0.346) and lowest EO (0.024), but at reduced accuracy.  

**Conclusion:** If the goal is **high accuracy with minimized error-rate disparity**, **GS(DP) i=14** is the strongest candidate.  
If **slightly better DP** is prioritized and accuracy loss is acceptable, **GS(DP) i=25** is preferable.  

----

### Bias Mitigation RD: Post-processing: Threshold Optimizer

In [22]:
from fairlearn.postprocessing import ThresholdOptimizer

# 0) Baseline RF 
rf.fit(X_train_ready, y_train)
y_rf_base = rf.predict(X_test_ready)
m_rf_base = eval_fairness(y_test, y_rf_base, A_test)

print("=== Baseline (Random Forest) ===")
print(m_rf_base["by_group"])
print(f"Accuracy: {m_rf_base['acc']:.4f} | DP diff: {m_rf_base['dp']:.4f} | EO diff: {m_rf_base['eo']:.4f}")

# 1) Post-processing: Equalized Odds 
post_rf_eo = ThresholdOptimizer(
    estimator=rf,
    constraints="equalized_odds",
    predict_method="predict_proba",   
    grid_size=200,
    flip=True
)
post_rf_eo.fit(X_train_ready, y_train, sensitive_features=A_train)
y_rf_eo = post_rf_eo.predict(X_test_ready, sensitive_features=A_test)
m_rf_eo = eval_fairness(y_test, y_rf_eo, A_test)

print("\n=== RF + Post-processing (Equalized Odds) ===")
print(m_rf_eo["by_group"])
print(f"Accuracy: {m_rf_eo['acc']:.4f} | DP diff: {m_rf_eo['dp']:.4f} | EO diff: {m_rf_eo['eo']:.4f}")

# 2) Post-processing: Demographic Parity 
post_rf_dp = ThresholdOptimizer(
    estimator=rf,
    constraints="demographic_parity",
    predict_method="predict_proba",
    grid_size=200,
    flip=True
)
post_rf_dp.fit(X_train_ready, y_train, sensitive_features=A_train)
y_rf_dp = post_rf_dp.predict(X_test_ready, sensitive_features=A_test)
m_rf_dp = eval_fairness(y_test, y_rf_dp, A_test)

print("\n=== RF + Post-processing (Demographic Parity) ===")
print(m_rf_dp["by_group"])
print(f"Accuracy: {m_rf_dp['acc']:.4f} | DP diff: {m_rf_dp['dp']:.4f} | EO diff: {m_rf_dp['eo']:.4f}")

#3) Summary Table
summary_rf_post = pd.DataFrame([
    {"model":"RF Baseline",       "accuracy":m_rf_base["acc"], "dp_diff":m_rf_base["dp"], "eo_diff":m_rf_base["eo"]},
    {"model":"RF + Post (EO)",    "accuracy":m_rf_eo["acc"],   "dp_diff":m_rf_eo["dp"],   "eo_diff":m_rf_eo["eo"]},
    {"model":"RF + Post (DP)",    "accuracy":m_rf_dp["acc"],   "dp_diff":m_rf_dp["dp"],   "eo_diff":m_rf_dp["eo"]},
]).round(4)

print("\n=== Random Forest: Baseline vs Post-processing ===")
print(summary_rf_post)

=== Baseline (Random Forest) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.833333  0.15625  0.833333       0.263158  0.842105
1    0.833333  0.18000  0.833333       0.609589  0.828767
Accuracy: 0.8315 | DP diff: 0.3464 | EO diff: 0.0237

=== RF + Post-processing (Equalized Odds) ===
          TPR  FPR    Recall  SelectionRate  Accuracy
Sex                                                  
0    0.833333  0.0  0.833333       0.131579  0.973684
1    0.833333  0.2  0.833333       0.616438  0.821918
Accuracy: 0.8533 | DP diff: 0.4849 | EO diff: 0.2000

=== RF + Post-processing (Demographic Parity) ===
          TPR  FPR    Recall  SelectionRate  Accuracy
Sex                                                  
0    0.833333  0.0  0.833333       0.131579  0.973684
1    0.833333  0.2  0.833333       0.616438  0.821918
Accuracy: 0.8533 | DP diff: 0.4849 | EO diff: 0.2000

=== Random Forest: Baseline vs Post-processin

# Random Forest Bias Mitigation (Post-processing)  

## Summary

| Model               | Accuracy | DP diff | EO diff | Interpretation                                      |
|---------------------|:--------:|:-------:|:-------:|-----------------------------------------------------|
| **RF Baseline**     | 0.8315   | 0.3464  | 0.0237  | Solid accuracy; moderate DP gap; EO very low.       |
| **RF + Post (EO)**  | 0.8533   | 0.4849  | 0.2000  | **Accuracy ↑**, but **DP worsens sharply**; EO ↑.   |
| **RF + Post (DP)**  | 0.8533   | 0.4849  | 0.2000  | Identical to EO → no fairness gain, disparities ↑.  |

---

## Interpretation
- **Baseline RF** already shows a **large DP disparity (0.35)** but **minimal EO (0.02)**.  
- **Post-processing (EO/DP)** increased accuracy slightly, but **both fairness gaps worsened**:  
  - **DP gap** rose from 0.346 → 0.485 due to much lower female selection (**0.13 vs 0.62** for males).  
  - **EO gap** widened from 0.024 → 0.200, with error-rate differences (**FPR 0.00 vs 0.20**).  
- Both **EO and DP constraints converged to the same adjusted predictions**, showing **no effective bias mitigation** here.  

**Takeaway:** For RF, **ThresholdOptimizer harms fairness** under both EO and DP settings—accuracy gains come at the cost of **much larger disparities**.  

---

### Deep Learning - Multi-layer Perceptron

In [23]:
#import required library 
from sklearn.neural_network import MLPClassifier

In [24]:
#  Improved MLP pipeline: recall-first tuning  
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import (
    f1_score, recall_score, fbeta_score, make_scorer
)


# 1) Recall-first search (Adam + early_stopping)
base_mlp = MLPClassifier(
    solver="adam",
    early_stopping=True,          # uses internal 15% validation
    validation_fraction=0.15,
    n_iter_no_change=20,
    max_iter=2000,                # allow convergence
    random_state=42
)

param_dist = {
    "hidden_layer_sizes": [(64,), (128,), (64, 32), (128, 64)],
    "activation": ["relu", "tanh"],
    "alpha": [1e-5, 1e-4, 3e-4, 1e-3],
    "learning_rate_init": [1e-3, 5e-4, 3e-4, 1e-4],
    "batch_size": [16, 32, 64],
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# multi-metric scoring; refit on recall-oriented F-beta
scoring = {
    "f1": make_scorer(f1_score),
    "recall": make_scorer(recall_score),
    "fbeta2": make_scorer(fbeta_score, beta=2)  # emphasize recall
}

rs = RandomizedSearchCV(
    estimator=base_mlp,
    param_distributions=param_dist,
    n_iter=30,
    scoring=scoring,
    refit="fbeta2",
    cv=cv,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

rs.fit(X_train_ready, y_train)
recallfirst_best_mlp = rs.best_estimator_
print("Best MLP params:", rs.best_params_)

# 2) Evaluation
y_prob = recallfirst_best_mlp.predict_proba(X_test_ready)[:, 1]
y_pred_best_mlp = recallfirst_best_mlp.predict(X_test_ready)
evaluate_model(y_test, y_pred_best_mlp, model_name=f"Best MLP")

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best MLP params: {'learning_rate_init': 0.001, 'hidden_layer_sizes': (64, 32), 'batch_size': 16, 'alpha': 0.001, 'activation': 'relu'}
=== Best MLP Evaluation ===
Accuracy : 0.8532608695652174
Precision: 0.9120879120879121
Recall   : 0.8137254901960784
F1 Score : 0.8601036269430051

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.90      0.85        82
           1       0.91      0.81      0.86       102

    accuracy                           0.85       184
   macro avg       0.85      0.86      0.85       184
weighted avg       0.86      0.85      0.85       184

Confusion Matrix:
 [[74  8]
 [19 83]]




### Bias mitigation MLP: Inprocessing: Exponentiated Gradient 

In [25]:
from sklearn.neural_network import MLPClassifier
from fairlearn.reductions import ExponentiatedGradient, EqualizedOdds, DemographicParity
from sklearn.base import clone
import pandas as pd


mlp = clone(recallfirst_best_mlp)

# 0) Baseline MLP (seeded for reproducibility)
mlp.fit(X_train_ready, y_train)

y_pred_mlp_base = mlp.predict(X_test_ready)
m_mlp_base = eval_fairness(y_test, y_pred_mlp_base, A_test)

print("=== Baseline (MLP) ===")
print(m_mlp_base["by_group"])
print(f"Accuracy: {m_mlp_base['acc']:.4f} | DP diff: {m_mlp_base['dp']:.4f} | EO diff: {m_mlp_base['eo']:.4f}")

# 1) EG with Equalized Odds
eg_eo_mlp = ExponentiatedGradient(
    estimator=clone(mlp),   # unfitted clone of your tuned MLP
    constraints=EqualizedOdds(),
    eps=0.01,
    max_iter=50
)
eg_eo_mlp.fit(X_train_ready, y_train, sensitive_features=A_train)

try:
    y_pred_mlp_eo = eg_eo_mlp.predict(X_test_ready, random_state=42)
except TypeError:
    y_pred_mlp_eo = eg_eo_mlp.predict(X_test_ready)

m_mlp_eo = eval_fairness(y_test, y_pred_mlp_eo, A_test)

print("\n=== In-processing MLP: EG (Equalized Odds) ===")
print(m_mlp_eo["by_group"])
print(f"Accuracy: {m_mlp_eo['acc']:.4f} | DP diff: {m_mlp_eo['dp']:.4f} | EO diff: {m_mlp_eo['eo']:.4f}")

# 2) EG with Demographic Parity
eg_dp_mlp = ExponentiatedGradient(
    estimator=clone(mlp),
    constraints=DemographicParity(),
    eps=0.01,
    max_iter=50
)
eg_dp_mlp.fit(X_train_ready, y_train, sensitive_features=A_train)

try:
    y_pred_mlp_dp = eg_dp_mlp.predict(X_test_ready, random_state=42)
except TypeError:
    y_pred_mlp_dp = eg_dp_mlp.predict(X_test_ready)

m_mlp_dp = eval_fairness(y_test, y_pred_mlp_dp, A_test)

print("\n=== In-processing MLP: EG (Demographic Parity) ===")
print(m_mlp_dp["by_group"])
print(f"Accuracy: {m_mlp_dp['acc']:.4f} | DP diff: {m_mlp_dp['dp']:.4f} | EO diff: {m_mlp_dp['eo']:.4f}")

# 3) Summary Table
summary_mlp = pd.DataFrame([
    {"model":"MLP Baseline",  "accuracy":m_mlp_base["acc"], "dp_diff":m_mlp_base["dp"], "eo_diff":m_mlp_base["eo"]},
    {"model":"MLP + EG (EO)", "accuracy":m_mlp_eo["acc"],   "dp_diff":m_mlp_eo["dp"],   "eo_diff":m_mlp_eo["eo"]},
    {"model":"MLP + EG (DP)", "accuracy":m_mlp_dp["acc"],   "dp_diff":m_mlp_dp["dp"],   "eo_diff":m_mlp_dp["eo"]},
]).round(4)

print("\n=== MLP: Baseline vs In-processing (EG) ===")
print(summary_mlp)

=== Baseline (MLP) ===
          TPR     FPR    Recall  SelectionRate  Accuracy
Sex                                                     
0    0.666667  0.0625  0.666667       0.157895  0.894737
1    0.822917  0.1200  0.822917       0.582192  0.842466
Accuracy: 0.8533 | DP diff: 0.4243 | EO diff: 0.1562

=== In-processing MLP: EG (Equalized Odds) ===
          TPR     FPR    Recall  SelectionRate  Accuracy
Sex                                                     
0    0.666667  0.0625  0.666667       0.157895  0.894737
1    0.770833  0.1800  0.770833       0.568493  0.787671
Accuracy: 0.8098 | DP diff: 0.4106 | EO diff: 0.1175

=== In-processing MLP: EG (Demographic Parity) ===
          TPR     FPR    Recall  SelectionRate  Accuracy
Sex                                                     
0    0.666667  0.0625  0.666667       0.157895  0.894737
1    0.791667  0.1600  0.791667       0.575342  0.808219
Accuracy: 0.8261 | DP diff: 0.4174 | EO diff: 0.1250

=== MLP: Baseline vs In-processin

### MLP — In-Processing: Exponentiated Gradient

#### Comparative summary
| Model          | Accuracy | ΔAcc (pp) | DP diff |  ΔDP   | EO diff |  ΔEO   | Notes |
|----------------|:--------:|:---------:|:-------:|:------:|:-------:|:------:|-------|
| **Baseline**   | 0.8533   |     –     | 0.4243  |   –    | 0.1562  |   –    | Reference |
| **EG (EO)**    | 0.8098   | **−4.4**  | 0.4106  | −0.0137| **0.1175**| **−0.0387** | Best EO improvement; small DP ↓; accuracy drops most |
| **EG (DP)**    | 0.8261   | −2.7      | 0.4174  | −0.0069| 0.1250  | −0.0312| Slight EO & DP gains; smaller accuracy hit |

#### Readout
- **Baseline:** Selection rates **S=0: 0.158** vs **S=1: 0.582** → **DP = 0.424**; TPR gap ≈ **0.156** dominates **EO = 0.156**.
- **EG (EO):** Narrows both **TPR** and **FPR** gaps → **EO ↓ to 0.118** and **DP ↓ slightly**, at a **−4.4 pp** accuracy cost.
- **EG (DP):** Also reduces **EO** (to **0.125**) and **DP** (to **0.417**), with a **smaller accuracy drop** (−2.7 pp).

**Takeaway:** Both EG variants improve fairness (EO ↓, DP ↓) but **trade off accuracy**.  
- If minimizing **error-rate disparity (EO)** is primary → **EG (EO)**.  
- If you want **milder accuracy loss** with still-better EO and DP → **EG (DP)**.  
DP remains sizeable in all cases; further tuning may be needed to close the selection-rate gap.

---

### Bias mitigation MLP: Inprocessing: Grid Search

In [26]:
from sklearn.base import clone
from fairlearn.reductions import GridSearch, EqualizedOdds, DemographicParity

mlp = clone(recallfirst_best_mlp)

# 1) GridSearch with Equalized Odds (MLP)
gs_eo_mlp = GridSearch(
    estimator=clone(mlp),                 # unfitted clone of your MLP (inherits random_state=42)
    constraints=EqualizedOdds(),
    selection_rule="tradeoff_optimization",  
    constraint_weight=0.5,                   
    grid_size=15                             
)
gs_eo_mlp.fit(X_train_ready, y_train, sensitive_features=A_train)
try:
    y_pred_gs_eo_mlp = gs_eo_mlp.predict(X_test_ready, random_state=42)
except TypeError:
    y_pred_gs_eo_mlp = gs_eo_mlp.predict(X_test_ready)

m_gs_eo_mlp = eval_fairness(y_test, y_pred_gs_eo_mlp, A_test)
print("\n=== In-processing MLP: GridSearch (Equalized Odds) ===")
print(m_gs_eo_mlp["by_group"])
print(f"Accuracy: {m_gs_eo_mlp['acc']:.4f} | DP diff: {m_gs_eo_mlp['dp']:.4f} | EO diff: {m_gs_eo_mlp['eo']:.4f}")

# 2) GridSearch with Demographic Parity (MLP)
gs_dp_mlp = GridSearch(
    estimator=clone(mlp),
    constraints=DemographicParity(),
    selection_rule="tradeoff_optimization",
    constraint_weight=0.5,
    grid_size=15
)
gs_dp_mlp.fit(X_train_ready, y_train, sensitive_features=A_train)
try:
    y_pred_gs_dp_mlp = gs_dp_mlp.predict(X_test_ready, random_state=42)
except TypeError:
    y_pred_gs_dp_mlp = gs_dp_mlp.predict(X_test_ready)

m_gs_dp_mlp = eval_fairness(y_test, y_pred_gs_dp_mlp, A_test)
print("\n=== In-processing MLP: GridSearch (Demographic Parity) ===")
print(m_gs_dp_mlp["by_group"])
print(f"Accuracy: {m_gs_dp_mlp['acc']:.4f} | DP diff: {m_gs_dp_mlp['dp']:.4f} | EO diff: {m_gs_dp_mlp['eo']:.4f}")

# 3) Compare with existing MLP runs (baseline + EG)
summary_mlp = pd.concat([
    summary_mlp,
    pd.DataFrame([
        {"model":"MLP + GS (EO)", "accuracy":m_gs_eo_mlp["acc"], "dp_diff":m_gs_eo_mlp["dp"], "eo_diff":m_gs_eo_mlp["eo"]},
        {"model":"MLP + GS (DP)", "accuracy":m_gs_dp_mlp["acc"], "dp_diff":m_gs_dp_mlp["dp"], "eo_diff":m_gs_dp_mlp["eo"]},
    ]).round(4)
], ignore_index=True)

print("\n=== MLP: Baseline vs EG vs GS ===")
print(summary_mlp)


=== In-processing MLP: GridSearch (Equalized Odds) ===
          TPR     FPR    Recall  SelectionRate  Accuracy
Sex                                                     
0    0.666667  0.0625  0.666667       0.157895  0.894737
1    0.822917  0.1200  0.822917       0.582192  0.842466
Accuracy: 0.8533 | DP diff: 0.4243 | EO diff: 0.1562

=== In-processing MLP: GridSearch (Demographic Parity) ===
          TPR     FPR    Recall  SelectionRate  Accuracy
Sex                                                     
0    0.666667  0.0625  0.666667       0.157895  0.894737
1    0.854167  0.2200  0.854167       0.636986  0.828767
Accuracy: 0.8424 | DP diff: 0.4791 | EO diff: 0.1875

=== MLP: Baseline vs EG vs GS ===
           model  accuracy  dp_diff  eo_diff
0   MLP Baseline    0.8533   0.4243   0.1562
1  MLP + EG (EO)    0.8098   0.4106   0.1175
2  MLP + EG (DP)    0.8261   0.4174   0.1250
3  MLP + GS (EO)    0.8533   0.4243   0.1562
4  MLP + GS (DP)    0.8424   0.4791   0.1875


### MLP — In-Processing: Exponentiated Gradient vs. GridSearch

#### Comparative Summary
| Model          | Accuracy | ΔAcc (pp) | DP diff |  ΔDP   | EO diff |  ΔEO   | Notes |
|----------------|:--------:|:---------:|:-------:|:------:|:-------:|:------:|-------|
| **Baseline**   | 0.8533   |    –      | 0.4243  |   –    | 0.1562  |   –    | Reference |
| **EG (EO)**    | 0.8098   | −4.4      | 0.4106  | −0.0137| **0.1175** | **−0.0387** | Best EO gain; small DP ↓; accuracy drops most |
| **EG (DP)**    | 0.8261   | −2.7      | 0.4174  | −0.0069| 0.1250  | −0.0312| Balanced: EO ↓, DP ↓ slightly; moderate accuracy hit |
| **GS (EO)**    | 0.8533   |   0.0     | 0.4243  |  0.0000| 0.1562  |  0.0000| Identical to baseline; constraint ineffective |
| **GS (DP)**    | 0.8424   | −1.1      | 0.4791  | +0.0548| 0.1875  | +0.0313| Accuracy ↓; DP and EO both **worsen** |

---

#### Interpretation
- **Baseline:** Already shows **large DP disparity (0.424)** and a moderate **EO gap (0.156)** due to different selection rates (Females 0.158 vs. Males 0.582).
- **Exponentiated Gradient (EG):**
  - **EG (EO):** Strongest fairness improvement → **EO ↓ to 0.118**, with a small DP reduction. However, comes with the **largest accuracy drop (−4.4 pp)**.
  - **EG (DP):** Produces a **milder trade-off**: both DP and EO improve slightly, accuracy loss smaller (−2.7 pp).
- **GridSearch (GS):**
  - **GS (EO):** No effect → converged to baseline solution.
  - **GS (DP):** **Counterproductive** → worsens both DP and EO, with reduced accuracy.

---

#### Takeaway
- **EG methods are effective** at improving fairness, especially **EG (EO)** for minimizing error-rate disparity, though at a cost to accuracy.
- **GS methods under current settings are ineffective or harmful**: GS (EO) had no effect, GS (DP) worsened disparities.
- For CVD bias mitigation, **EG (EO)** is best if **minimizing EO** is the priority; **EG (DP)** offers a more balanced option with smaller accuracy loss.

---

### Bias mitigation MLP: Postprocessing: Threshold Optimizer

In [27]:
from fairlearn.postprocessing import ThresholdOptimizer
import pandas as pd

mlp = clone(recallfirst_best_mlp)

# 0) Baseline MLP
mlp.fit(X_train_ready, y_train)
y_mlp_base = mlp.predict(X_test_ready)
m_mlp_base = eval_fairness(y_test, y_mlp_base, A_test)

print("=== Baseline (MLP) ===")
print(m_mlp_base["by_group"])
print(f"Accuracy: {m_mlp_base['acc']:.4f} | DP diff: {m_mlp_base['dp']:.4f} | EO diff: {m_mlp_base['eo']:.4f}")

# 1) Post-processing: Equalized Odds
post_mlp_eo = ThresholdOptimizer(
    estimator=mlp,
    constraints="equalized_odds",
    predict_method="predict_proba",
    grid_size=200,
    flip=True
)
post_mlp_eo.fit(X_train_ready, y_train, sensitive_features=A_train)
y_mlp_eo = post_mlp_eo.predict(X_test_ready, sensitive_features=A_test)
m_mlp_eo = eval_fairness(y_test, y_mlp_eo, A_test)

print("\n=== MLP + Post-processing (Equalized Odds) ===")
print(m_mlp_eo["by_group"])
print(f"Accuracy: {m_mlp_eo['acc']:.4f} | DP diff: {m_mlp_eo['dp']:.4f} | EO diff: {m_mlp_eo['eo']:.4f}")

# 2) Post-processing: Demographic Parity
post_mlp_dp = ThresholdOptimizer(
    estimator=mlp,
    constraints="demographic_parity",
    predict_method="predict_proba",
    grid_size=200,
    flip=True
)
post_mlp_dp.fit(X_train_ready, y_train, sensitive_features=A_train)
y_mlp_dp = post_mlp_dp.predict(X_test_ready, sensitive_features=A_test)
m_mlp_dp = eval_fairness(y_test, y_mlp_dp, A_test)

print("\n=== MLP + Post-processing (Demographic Parity) ===")
print(m_mlp_dp["by_group"])
print(f"Accuracy: {m_mlp_dp['acc']:.4f} | DP diff: {m_mlp_dp['dp']:.4f} | EO diff: {m_mlp_dp['eo']:.4f}")

# 3) Summary Table
summary_mlp_post = pd.DataFrame([
    {"model":"MLP Baseline",       "accuracy":m_mlp_base["acc"], "dp_diff":m_mlp_base["dp"], "eo_diff":m_mlp_base["eo"]},
    {"model":"MLP + Post (EO)",    "accuracy":m_mlp_eo["acc"],   "dp_diff":m_mlp_eo["dp"],   "eo_diff":m_mlp_eo["eo"]},
    {"model":"MLP + Post (DP)",    "accuracy":m_mlp_dp["acc"],   "dp_diff":m_mlp_dp["dp"],   "eo_diff":m_mlp_dp["eo"]},
]).round(4)

print("\n=== MLP: Baseline vs Post-processing ===")
print(summary_mlp_post)

=== Baseline (MLP) ===
          TPR     FPR    Recall  SelectionRate  Accuracy
Sex                                                     
0    0.666667  0.0625  0.666667       0.157895  0.894737
1    0.822917  0.1200  0.822917       0.582192  0.842466
Accuracy: 0.8533 | DP diff: 0.4243 | EO diff: 0.1562

=== MLP + Post-processing (Equalized Odds) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.666667  0.09375  0.666667       0.184211  0.868421
1    0.822917  0.12000  0.822917       0.582192  0.842466
Accuracy: 0.8478 | DP diff: 0.3980 | EO diff: 0.1562

=== MLP + Post-processing (Demographic Parity) ===
          TPR     FPR    Recall  SelectionRate  Accuracy
Sex                                                     
0    0.666667  0.0625  0.666667       0.157895  0.894737
1    0.875000  0.2200  0.875000       0.650685  0.842466
Accuracy: 0.8533 | DP diff: 0.4928 | EO diff: 0.2083

=== MLP: Baseline vs Post-pro

# Gender Bias Mitigation in CVD Prediction — Overall Interpretation

**Fairness metrics:**  
- **DP diff** (Demographic Parity): selection-rate gap across sexes (lower = more equal triage/alerts).  
- **EO diff** (Equalized Odds): error-rate gap (combined TPR/FPR gap) across sexes (lower = more equal misses/false alarms).

---

## One-glance summary (best observed per family)

| Model family | Best config in your runs | Accuracy | DP diff | EO diff | Why it’s “best” |
|---|---:|---:|---:|---:|---|
| **PCA+KNN** | *(none effective)* | — | — | — | Post-processing made **0% flips**; CR modestly ↓DP/EO but at an accuracy cost. |
| **Decision Tree** | **EG (DP)** | 0.7826 | **0.2812** | **0.0208** | Strong EO gain, DP reduced, with modest acc cost. |
|  | Post (EO) | 0.8315 | 0.3327 | 0.1667 | Accuracy ↑, but fairness worsened. |
|  | Post (DP) | 0.8424 | 0.4380 | 0.3333 | Accuracy ↑, but both DP & EO worsened. |
| **Random Forest** | **GS (DP, i=14)** | **0.8750** | 0.3991 | **0.0312** | Highest accuracy among RF; EO strongly improved; DP still large. |
|  | GS (DP, i=25) | 0.8315 | **0.3464** | **0.0237** | Matches baseline acc; lowest DP/EO combo but no gains. |
| **MLP** | **EG (EO)** | 0.8098 | 0.4106 | **0.1175** | Best EO reduction (−0.039); acc −4.4 pp. |
|  | **EG (DP)** | 0.8261 | 0.4174 | 0.1250 | Balanced EO/DP ↓; acc −2.7 pp. |
|  | *(Post / GS)* | — | — | — | Post-processing and GS were **harmful or ineffective**. |

> Baselines to remember: **DT** (acc 0.8207, DP 0.3306, EO 0.1042) • **RF** (acc 0.8315, DP 0.346, EO 0.024) • **MLP** (acc 0.8533, DP 0.424, EO 0.156) • **PCA+KNN** (acc 0.875, DP 0.438, EO 0.188)

---

## What worked 

### When the clinical priority is **error-rate parity (EO)**  

- **Decision Tree + EG (DP):** EO **0.104 → 0.021**, DP ↓, accuracy −3.8 pp.  
- **Random Forest + GS (DP, i=14):** EO **0.024 → 0.031** with **highest acc (0.875)**.  
- **MLP + EG (EO):** EO **0.156 → 0.118**; strongest EO improvement for NN, but accuracy −4.4 pp.  

**Pick:**  
- For **near-zero EO** → **DT + EG (DP)**.  
- For **best EO in an ensemble** → **RF + GS (DP, i=14)**.  
- For **NN fairness** → **MLP + EG (EO)**, if accuracy loss acceptable.  

---

### When the priority is **outcome parity (DP)**  

- **Decision Tree + EG (DP):** DP **0.331 → 0.281** (improvement), EO nearly eliminated, modest acc drop.  
- **Random Forest + GS (DP, i=25):** Best DP (0.346) with EO 0.024, but no acc gain.  
- **MLP + EG (DP):** DP **0.424 → 0.417**, EO ↓, with moderate acc drop.  

**Pick:**  
- For **DT** → **EG (DP)** (best DP/EO combo).  
- For **RF** → **GS (DP, i=25)** (best DP/EO at baseline acc).  
- For **NN** → **EG (DP)** (slight DP/EO ↓ with moderate acc hit).  

---

### Approaches that were **ineffective or harmful**

- **PCA+KNN Post-processing:** **0% label flips**; CR improved fairness slightly but reduced acc.  
- **RF EG (EO/DP):** **No effect**.  
- **RF Post (EO/DP):** Acc ↑ but **DP and EO worsened sharply**.  
- **MLP GS & Post-processing:** Mostly worsened fairness or accuracy.  
- **DT Post (EO/DP):** Accuracy ↑ but **both DP and EO gaps worsened**.  
- **DT GS (EO):** No change (baseline repeated).  
- **DT GS (DP):** Minor DP ↓, but EO ↑ and accuracy ↓.  
- **RF GS (EO):** Generated **degenerate solutions** (low acc, EO explosion).  

---

### Practical guidance for CVD deployment

1. **Decide your fairness target:**  
   - **EO focus (clinical safety):** minimize error-rate disparity → aim EO ≤ 0.05.  
   - **DP focus (access equity):** equalize selection rates → aim DP ≤ 0.10–0.15.  

2. **Pick mitigation accordingly:**  
   - **EO-driven:**  
     - **DT + EG (DP):** EO nearly eliminated (0.021).  
     - **RF + GS (DP, i=14):** EO 0.031, high acc (0.875).  
     - **MLP + EG (EO):** EO 0.118, tolerable acc loss.  
   - **DP-driven:**  
     - **DT + EG (DP):** DP 0.281, EO 0.021.  
     - **RF + GS (DP, i=25):** lowest DP (0.346) with EO 0.024.  
     - **MLP + EG (DP):** mild DP/EO ↓ with moderate acc loss.  

3. **Lock criteria before final selection:**  
   - Use thresholds (e.g., **EO ≤ 0.05, DP ≤ 0.30, acc ≥ baseline −0.5 pp**).  
   - Select frontier candidates that meet all thresholds.  

---

### Summary

- **No universal fix**: each model family offers different trade-offs.  
- **EO parity** (error-rate balance) is best addressed by:  
  - **DT + EG (DP)** (EO ↓ to 0.021),  
  - **RF + GS (DP, i=14)** (EO 0.031 with top accuracy),  
  - **MLP + EG (EO)** (EO ↓ with accuracy hit).  
- **DP parity** (equal outcomes) is harder: only **DT + EG (DP)** achieved a meaningful DP ↓ (0.281).  
- **Avoid** PCA+KNN post, RF EG/Post, DT Post, DT GS(EO), and MLP GS/Post — they failed to improve fairness.  

---